# LIBRARIES

In [18]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import random

# 1. Initial Setup and Connection to BigQuery


In [21]:

## construct credentials from service account key file
credentials = service_account.Credentials.from_service_account_file(
    'C:\\Users\\ivani\\Documents\\Emory University\\Spring 2025\\Machine Learning At Scale\\iumuhoz_bq (1)\\iumuhoz-isom676-srvacct.json') ## relative file path
## '/mnt/c/Users/PGU6/workspace-GBS/student-technology-tools/docs/gcp/.ssl/bq_srv_acct.json') ## absolute file path

## construct a BigQuery client object
client = bigquery.Client(credentials=credentials)

## Your logics implementation goes below
QUERY = (
    'SELECT prod_id, prod_desc FROM `msba-emory.isom676_machine_learning.products` '
    'WHERE prod_id = 20971208 '
    'LIMIT 5')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row[0]) ## by index
    print(row.prod_id) ## by column prod_id
    print(row.prod_desc) ## by column prod_id

print("Connect and query BigQuery successfully!")

RefreshError: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})

# 2. Stratified Sampling of Transactions Data

In [ ]:
# Let's first get the list of stores to use for stratification
STORES_QUERY = """
SELECT DISTINCT store_id
FROM `msba-emory.isom676_machine_learning.transactions`
"""

print("Retrieving list of stores...")
stores = client.query(STORES_QUERY).to_dataframe()
store_ids = stores['store_id'].tolist()
print(f"Found {len(store_ids)} stores")

# sample a  small percentage from each store
# 0.1% should be manageable while still giving enough data
sample_fraction = 0.001  # 0.1%
print(f"Will sample {sample_fraction*100:.2f}% from each store")

# Create an empty dataframe to hold our sample
transactions_sample = pd.DataFrame()

# Sample from each store separately using a simpler approach
for i, store_id in enumerate(store_ids):
    print(f"Sampling from store {store_id} ({i+1}/{len(store_ids)})...")
    
    # First, count the number of transactions for this store
    COUNT_QUERY = f"""
    SELECT COUNT(*) as count
    FROM `msba-emory.isom676_machine_learning.transactions`
    WHERE store_id = {store_id}
    """
    
    count_result = client.query(COUNT_QUERY).to_dataframe()
    store_count = count_result['count'].iloc[0]
    
    # Calculate sample size for this store
    sample_size = max(1, int(store_count * sample_fraction))
    
    # Now sample from this store
    STORE_SAMPLE_QUERY = f"""
    SELECT 
        cust_id,
        store_id,
        prod_id,
        trans_id,
        trans_dt,
        sales_qty,
        sales_wgt,
        sales_amt
    FROM `msba-emory.isom676_machine_learning.transactions`
    WHERE store_id = {store_id}
    ORDER BY RAND()
    LIMIT {sample_size}
    """
    
    try:
        store_sample = client.query(STORE_SAMPLE_QUERY).to_dataframe()
        print(f"  - Sampled {len(store_sample)} transactions from store {store_id}")
        
        # Add to our combined sample
        transactions_sample = pd.concat([transactions_sample, store_sample])
    except Exception as e:
        print(f"  - Error sampling store {store_id}: {str(e)}")

print(f"\nFinished sampling. Total sampled transactions: {len(transactions_sample):,}")

# Convert date column to datetime
transactions_sample['trans_dt'] = pd.to_datetime(transactions_sample['trans_dt'])

# Save sampled transactions for later use
transactions_sample.to_csv('transactions_sample.csv', index=False)

# 3.  Retrieve Full Products Dataset

In [ ]:
# Get the complete products dataset (no sampling needed)
PRODUCTS_QUERY = """
SELECT *
FROM `msba-emory.isom676_machine_learning.products`
"""

print("\nRetrieving full products dataset...")
products = client.query(PRODUCTS_QUERY).to_dataframe()
print(f"Retrieved {len(products):,} products")

# Save products data for later use
products.to_csv('products.csv', index=False)

# 4. Initial Data Exploration and Quality Assessment

In [ ]:
# Load data from saved files (if already saved)
try:
    transactions_sample = pd.read_csv('transactions_sample.csv')
    transactions_sample['trans_dt'] = pd.to_datetime(transactions_sample['trans_dt'])
    products = pd.read_csv('products.csv')
    print("Loaded data from saved CSV files.")
except:
    print("Using data from previous steps.")

print("\n=== Initial Data Exploration ===")

# Basic information about the datasets
print("\nTransactions Sample Overview:")
print(f"• Number of transactions: {len(transactions_sample):,}")
print(f"• Number of unique customers: {transactions_sample['cust_id'].nunique():,}")
print(f"• Number of unique stores: {transactions_sample['store_id'].nunique():,}")
print(f"• Number of unique products: {transactions_sample['prod_id'].nunique():,}")
print(f"• Date range: {transactions_sample['trans_dt'].min().date()} to {transactions_sample['trans_dt'].max().date()}")
print(f"• Total sales amount: ${transactions_sample['sales_amt'].sum():,.2f}")

print("\nTransactions Data Types:")
print(transactions_sample.dtypes)

print("\nTransactions Summary Statistics:")
print(transactions_sample.describe().round(2))

print("\nProducts Dataset Overview:")
print(f"• Number of products: {len(products):,}")
if 'prod_category' in products.columns:
    print(f"• Number of categories: {products['prod_category'].nunique():,}")
if 'prod_subcategory' in products.columns:
    print(f"• Number of subcategories: {products['prod_subcategory'].nunique():,}")
if 'prod_section' in products.columns:
    print(f"• Number of sections: {products['prod_section'].nunique():,}")

print("\nProducts Data Types:")
print(products.dtypes)

# Check for missing values in both datasets
print("\n=== Missing Values Analysis ===")
trans_missing = transactions_sample.isnull().sum()
prod_missing = products.isnull().sum()

print("\nTransactions - Missing values:")
if trans_missing.sum() > 0:
    for col, count in trans_missing[trans_missing > 0].items():
        print(f"• Column '{col}': {count:,} missing values ({count/len(transactions_sample)*100:.2f}%)")
else:
    print("No missing values found in transactions data.")

print("\nProducts - Missing values:")
if prod_missing.sum() > 0:
    for col, count in prod_missing[prod_missing > 0].items():
        print(f"• Column '{col}': {count:,} missing values ({count/len(products)*100:.2f}%)")
else:
    print("No missing values found in products data.")

# Store pre-cleaning metrics for later comparison
original_missing_count = transactions_sample.isnull().sum().sum() + products.isnull().sum().sum()
original_neg_sales_count = len(transactions_sample[transactions_sample['sales_amt'] < 0])
original_zero_neg_qty_count = len(transactions_sample[transactions_sample['sales_qty'] <= 0])

print(f"\nPre-cleaning metrics:")
print(f"• Total missing values: {original_missing_count:,}")
print(f"• Negative sales transactions: {original_neg_sales_count:,} ({original_neg_sales_count/len(transactions_sample)*100:.2f}%)")
print(f"• Zero/negative quantity transactions: {original_zero_neg_qty_count:,} ({original_zero_neg_qty_count/len(transactions_sample)*100:.2f}%)")

# 5. Pre-Cleaning Visualizations

In [ ]:
print("\n=== Creating Pre-Cleaning Visualizations ===")

# 1. Distribution of sales amounts
plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
sns.histplot(transactions_sample['sales_amt'].clip(upper=transactions_sample['sales_amt'].quantile(0.99)), 
             bins=50, kde=True, color='blue')
plt.title('Sales Amount Distribution (99th Percentile)\nBefore Cleaning', fontsize=14)
plt.xlabel('Sales Amount ($)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Sales amount boxplot
plt.subplot(1, 2, 2)
sns.boxplot(y=transactions_sample['sales_amt'].clip(upper=transactions_sample['sales_amt'].quantile(0.99)), color='blue')
plt.title('Sales Amount Boxplot (99th Percentile)\nBefore Cleaning', fontsize=14)
plt.ylabel('Sales Amount ($)', fontsize=12)
plt.tight_layout()
plt.savefig('visualizations/before_cleaning_sales_distribution.png')
plt.show()

# 2. Distribution of sales quantities
plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
sns.histplot(transactions_sample['sales_qty'].clip(upper=transactions_sample['sales_qty'].quantile(0.99)), 
             bins=50, kde=True, color='green')
plt.title('Sales Quantity Distribution (99th Percentile)\nBefore Cleaning', fontsize=14)
plt.xlabel('Sales Quantity', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Sales quantity boxplot
plt.subplot(1, 2, 2)
sns.boxplot(y=transactions_sample['sales_qty'].clip(upper=transactions_sample['sales_qty'].quantile(0.99)), color='green')
plt.title('Sales Quantity Boxplot (99th Percentile)\nBefore Cleaning', fontsize=14)
plt.ylabel('Sales Quantity', fontsize=12)
plt.tight_layout()
plt.savefig('visualizations/before_cleaning_quantity_distribution.png')
plt.show()

# 3. Missing values heatmap
plt.figure(figsize=(14, 6))
transactions_missing = transactions_sample.isnull()
products_missing = products.isnull()

plt.subplot(1, 2, 1)
sns.heatmap(transactions_missing.sample(min(1000, len(transactions_missing))), 
            cbar=False, cmap='viridis', yticklabels=False)
plt.title('Missing Values in Transactions (Sample)\nBefore Cleaning', fontsize=14)

plt.subplot(1, 2, 2)
sns.heatmap(products_missing.sample(min(1000, len(products_missing))), 
            cbar=False, cmap='viridis', yticklabels=False)
plt.title('Missing Values in Products (Sample)\nBefore Cleaning', fontsize=14)
plt.tight_layout()
plt.savefig('visualizations/before_cleaning_missing_values.png')
plt.show()

# 4. Negative sales amounts (if any)
neg_sales = transactions_sample[transactions_sample['sales_amt'] < 0]
if len(neg_sales) > 0:
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(neg_sales['sales_amt'], bins=30, kde=True, color='red')
    plt.title('Distribution of Negative Sales Amounts\nBefore Cleaning', fontsize=14)
    plt.xlabel('Sales Amount ($)', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    
    plt.subplot(1, 2, 2)
    monthly_returns = neg_sales.groupby(neg_sales['trans_dt'].dt.to_period('M'))['sales_amt'].sum()
    monthly_returns = monthly_returns.reset_index()
    monthly_returns['trans_dt'] = monthly_returns['trans_dt'].astype(str)
    plt.bar(monthly_returns['trans_dt'], monthly_returns['sales_amt'].abs(), color='red')
    plt.title('Monthly Return Amounts\nBefore Cleaning', fontsize=14)
    plt.xlabel('Month', fontsize=12)
    plt.ylabel('Total Returns ($)', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('visualizations/before_cleaning_negative_sales.png')
    plt.show()

# 5. Zero or negative quantities (if any)
zero_neg_qty = transactions_sample[transactions_sample['sales_qty'] <= 0]
if len(zero_neg_qty) > 0:
    plt.figure(figsize=(12, 6))
    # Create a count by value for the most common zero/negative quantities
    qty_counts = zero_neg_qty['sales_qty'].value_counts().sort_index().head(10)
    plt.bar(qty_counts.index.astype(str), qty_counts.values, color='purple')
    plt.title('Count of Zero or Negative Quantities\nBefore Cleaning', fontsize=14)
    plt.xlabel('Sales Quantity', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.tight_layout()
    plt.savefig('visualizations/before_cleaning_zero_negative_qty.png')
    plt.show()

# 6. Sales over time
plt.figure(figsize=(14, 7))
# Group by day and calculate total sales
daily_sales = transactions_sample.groupby(transactions_sample['trans_dt'].dt.date)['sales_amt'].sum()
plt.plot(daily_sales.index, daily_sales.values, color='blue', linewidth=2)
plt.title('Daily Sales Over Time\nBefore Cleaning', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Total Sales Amount ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('visualizations/before_cleaning_daily_sales.png')
plt.show()

print("Pre-cleaning visualizations created and displayed")

# 6. Data Cleaning and Anomaly Detection

In [ ]:
print("\n=== Data Cleaning and Anomaly Detection ===")

# Create copies of the data for cleaning
cleaned_trans = transactions_sample.copy()
cleaned_prods = products.copy()

# 1. Handle missing values
print("\n--- Handling Missing Values ---")

# For transactions
if trans_missing.sum() > 0:
    for col in trans_missing[trans_missing > 0].index:
        if col == 'sales_qty':
            cleaned_trans['sales_qty'] = cleaned_trans['sales_qty'].fillna(1)
            print(f"• Filled {trans_missing[col]:,} missing values in 'sales_qty' with default value 1")
            print("  Rationale: In retail, a missing quantity most commonly represents a single item purchase. Using 1 as default preserves transaction integrity.")
        elif col == 'sales_wgt':
            cleaned_trans['sales_wgt'] = cleaned_trans['sales_wgt'].fillna(0)
            print(f"• Filled {trans_missing[col]:,} missing values in 'sales_wgt' with default value 0")
            print("  Rationale: Missing weight values likely indicate non-weight-based products. Zero is appropriate as these products aren't sold by weight.")
        elif col == 'sales_amt':
            # For missing sales amounts, use the median price for that product
            missing_amt_count = cleaned_trans['sales_amt'].isnull().sum()
            for prod_id in cleaned_trans[cleaned_trans['sales_amt'].isnull()]['prod_id'].unique():
                # Get median price per unit for this product
                prod_trans = cleaned_trans[(cleaned_trans['prod_id'] == prod_id) & 
                                          cleaned_trans['sales_amt'].notnull() & 
                                          (cleaned_trans['sales_qty'] > 0)]
                if len(prod_trans) > 0:
                    median_price_per_unit = (prod_trans['sales_amt'] / prod_trans['sales_qty']).median()
                    # Apply to missing values
                    mask = (cleaned_trans['prod_id'] == prod_id) & cleaned_trans['sales_amt'].isnull()
                    cleaned_trans.loc[mask, 'sales_amt'] = cleaned_trans.loc[mask, 'sales_qty'] * median_price_per_unit
            
            # For any remaining missing values, use overall median price
            remaining_missing = cleaned_trans['sales_amt'].isnull().sum()
            if remaining_missing > 0:
                overall_median = cleaned_trans[cleaned_trans['sales_amt'].notnull()]['sales_amt'].median()
                cleaned_trans['sales_amt'] = cleaned_trans['sales_amt'].fillna(overall_median)
                print(f"  Note: Used global median price for {remaining_missing:,} transactions where product-specific data was unavailable")
            
            print(f"• Filled {missing_amt_count:,} missing values in 'sales_amt' using product-specific median prices")
            print("  Method: Calculated median price per unit for each product, then multiplied by quantity")
            print("  Rationale: Product-specific pricing maintains data accuracy by respecting the unique price point of each product, rather than using a global value")
else:
    print("No missing values in transactions data to handle.")

# For products
if prod_missing.sum() > 0:
    for col in prod_missing[prod_missing > 0].index:
        if col == 'prod_desc':
            cleaned_prods['prod_desc'] = cleaned_prods['prod_desc'].fillna("MISSING DESCRIPTION")
            print(f"• Filled {prod_missing[col]:,} missing values in 'prod_desc' with 'MISSING DESCRIPTION'")
            print("  Rationale: Using a distinctive placeholder allows easy identification of these products for further investigation while maintaining data completeness")
        else:
            cleaned_prods[col] = cleaned_prods[col].fillna("Unknown")
            print(f"• Filled {prod_missing[col]:,} missing values in '{col}' with 'Unknown'")
            print("  Method: Category imputation with standard placeholder")
            print("  Rationale: Preserves these products in the dataset while clearly indicating missing information")
else:
    print("No missing values in products data to handle.")

# 2. Text standardization
print("\n--- Standardizing Text Fields ---")
if 'prod_desc' in cleaned_prods.columns:
    cleaned_prods['prod_desc'] = cleaned_prods['prod_desc'].astype(str).str.strip().str.lower()
    print("• Standardized product descriptions (lowercase, trimmed spaces)")
    print("  Method: Applied string transformation functions (strip and lowercase)")
    print("  Rationale: Ensures consistent text formatting for accurate matching and analysis, eliminating variations due to capitalization and whitespace")

if 'prod_category' in cleaned_prods.columns:
    cleaned_prods['prod_category'] = cleaned_prods['prod_category'].astype(str).str.strip()
    print("• Standardized product categories (trimmed spaces)")
    print("  Rationale: Category names need consistent formatting for correct grouping in analyses")

if 'prod_subcategory' in cleaned_prods.columns:
    cleaned_prods['prod_subcategory'] = cleaned_prods['prod_subcategory'].astype(str).str.strip()
    print("• Standardized product subcategories (trimmed spaces)")
    print("  Rationale: Ensures consistent hierarchical categorization and prevents duplicate categories due to whitespace variations")

# 3. Handle anomalies
print("\n--- Detecting and Handling Anomalies ---")

# 3.1 Identify and flag returns (negative sales)
neg_sales = cleaned_trans[cleaned_trans['sales_amt'] < 0]
if len(neg_sales) > 0:
    cleaned_trans['is_return'] = cleaned_trans['sales_amt'] < 0
    print(f"• Identified and flagged {len(neg_sales):,} transactions as returns (negative sales)")
    print(f"  Average return amount: ${neg_sales['sales_amt'].mean():.2f}")
    print("  Method: Created boolean flag column 'is_return' instead of removing these records")
    print("  Rationale: Negative sales values likely represent valid returns or refunds - removing them would artificially inflate sales metrics, while flagging allows separate analysis of return patterns")

# 3.2 Fix zero or negative quantities with positive sales
zero_with_sales = (cleaned_trans['sales_qty'] <= 0) & (cleaned_trans['sales_amt'] > 0)
if zero_with_sales.sum() > 0:
    cleaned_trans.loc[zero_with_sales, 'sales_qty'] = 1
    print(f"• Fixed {zero_with_sales.sum():,} transactions with zero/negative quantities but positive sales")
    print("  Method: Replaced illogical values with the minimum valid quantity (1)")
    print("  Rationale: These are likely data entry errors since a transaction cannot have a positive sales amount with zero or negative quantity. Setting to 1 preserves the transaction while resolving the logical inconsistency.")

# 3.3 Identify outlier customers
# Calculate customer metrics
customer_stats = cleaned_trans.groupby('cust_id').agg({
    'sales_amt': 'sum',
    'trans_id': 'nunique',
    'prod_id': 'nunique'
})
customer_stats.columns = ['total_spent', 'num_transactions', 'unique_products']

# Use Z-score method to identify outliers (3 standard deviations)
outlier_customers = set()
for col in ['total_spent', 'num_transactions', 'unique_products']:
    mean = customer_stats[col].mean()
    std = customer_stats[col].std()
    threshold = mean + 3 * std
    outliers = customer_stats[customer_stats[col] > threshold]
    
    if len(outliers) > 0:
        print(f"• Found {len(outliers):,} outlier customers based on {col}")
        print(f"  Threshold: {threshold:.2f}, Max value: {outliers[col].max():.2f}")
        outlier_customers.update(outliers.index)

# Flag outlier customers in the transactions
cleaned_trans['is_outlier_customer'] = cleaned_trans['cust_id'].isin(outlier_customers)
print(f"• Flagged {len(outlier_customers):,} unique outlier customers in the dataset")
print("  Method: Z-score outlier detection with threshold of 3 standard deviations")
print("  Rationale: Using statistical thresholds identifies true outliers while preserving their data for analysis. These could represent high-value customers, business accounts, or potential data errors that warrant special attention.")

# 3.4 Identify stores with unusual transaction counts
store_stats = cleaned_trans.groupby('store_id').agg({
    'trans_id': 'nunique',
    'cust_id': 'nunique'
})
store_stats.columns = ['num_transactions', 'num_customers']

# Find stores with too few transactions (bottom 5%)
low_threshold = np.percentile(store_stats['num_transactions'], 5)
low_activity_stores = store_stats[store_stats['num_transactions'] < low_threshold]

if len(low_activity_stores) > 0:
    cleaned_trans['is_low_activity_store'] = cleaned_trans['store_id'].isin(low_activity_stores.index)
    print(f"• Identified {len(low_activity_stores):,} stores with unusually low transaction counts")
    print(f"  These stores have fewer than {low_threshold:.0f} transactions")
    print("  Method: Percentile-based anomaly detection (bottom 5%)")
    print("  Rationale: These stores may be new locations, closing stores, or have data collection issues. Flagging allows for targeted analysis without removing potentially valid transactions.")

# 3.5 Check for inconsistencies between transactions and products
trans_products = set(cleaned_trans['prod_id'].unique())
catalog_products = set(cleaned_prods['prod_id'].unique())

missing_product_info = trans_products - catalog_products
if len(missing_product_info) > 0:
    cleaned_trans['has_missing_product_info'] = cleaned_trans['prod_id'].isin(missing_product_info)
    print(f"• Found {len(missing_product_info):,} products in transactions that aren't in the product catalog")
    
    # Count affected transactions
    affected_trans = cleaned_trans[cleaned_trans['has_missing_product_info']]
    print(f"  This affects {len(affected_trans):,} transactions ({len(affected_trans)/len(cleaned_trans)*100:.2f}% of sample)")
    print("  Method: Created flag column to identify these transactions rather than removing them")
    print("  Rationale: These may be discontinued products, new products not yet in the catalog, or data entry errors. Flagging preserves transaction data while highlighting the inconsistency for further investigation.")

# Save cleaned data
cleaned_trans.to_csv('transactions_cleaned.csv', index=False)
cleaned_prods.to_csv('products_cleaned.csv', index=False)

print("\nData cleaning complete! Cleaned data saved to CSV files.")
print("Cleaning approach: Conservative data preservation with explicit flagging of anomalies")
print("Key principle: Maintained as much original data as possible while resolving inconsistencies and flagging unusual patterns for further analysis")

# 7. Post-Cleaning Visualizations

In [ ]:
print("\n=== Creating Post-Cleaning Visualizations ===")

# 1. Distribution of sales amounts after cleaning
plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
sns.histplot(cleaned_trans['sales_amt'].clip(upper=cleaned_trans['sales_amt'].quantile(0.99)), 
             bins=50, kde=True, color='blue')
plt.title('Sales Amount Distribution (99th Percentile)\nAfter Cleaning', fontsize=14)
plt.xlabel('Sales Amount ($)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Sales amount boxplot after cleaning
plt.subplot(1, 2, 2)
sns.boxplot(y=cleaned_trans['sales_amt'].clip(upper=cleaned_trans['sales_amt'].quantile(0.99)), color='blue')
plt.title('Sales Amount Boxplot (99th Percentile)\nAfter Cleaning', fontsize=14)
plt.ylabel('Sales Amount ($)', fontsize=12)
plt.tight_layout()
plt.savefig('visualizations/after_cleaning_sales_distribution.png')
plt.show()

# 2. Distribution of sales quantities after cleaning
plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
sns.histplot(cleaned_trans['sales_qty'].clip(upper=cleaned_trans['sales_qty'].quantile(0.99)), 
             bins=50, kde=True, color='green')
plt.title('Sales Quantity Distribution (99th Percentile)\nAfter Cleaning', fontsize=14)
plt.xlabel('Sales Quantity', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Sales quantity boxplot after cleaning
plt.subplot(1, 2, 2)
sns.boxplot(y=cleaned_trans['sales_qty'].clip(upper=cleaned_trans['sales_qty'].quantile(0.99)), color='green')
plt.title('Sales Quantity Boxplot (99th Percentile)\nAfter Cleaning', fontsize=14)
plt.ylabel('Sales Quantity', fontsize=12)
plt.tight_layout()
plt.savefig('visualizations/after_cleaning_quantity_distribution.png')
plt.show()

# 3. Missing values heatmap after cleaning
plt.figure(figsize=(14, 6))
cleaned_trans_missing = cleaned_trans.isnull()
cleaned_prods_missing = cleaned_prods.isnull()

plt.subplot(1, 2, 1)
sns.heatmap(cleaned_trans_missing.sample(min(1000, len(cleaned_trans_missing))), 
            cbar=False, cmap='viridis', yticklabels=False)
plt.title('Missing Values in Transactions (Sample)\nAfter Cleaning', fontsize=14)

plt.subplot(1, 2, 2)
sns.heatmap(cleaned_prods_missing.sample(min(1000, len(cleaned_prods_missing))), 
            cbar=False, cmap='viridis', yticklabels=False)
plt.title('Missing Values in Products (Sample)\nAfter Cleaning', fontsize=14)
plt.tight_layout()
plt.savefig('visualizations/after_cleaning_missing_values.png')
plt.show()

# 4. Before vs After Comparison
# Calculate current metrics
current_missing_count = cleaned_trans.isnull().sum().sum() + cleaned_prods.isnull().sum().sum()
current_neg_sales_count = len(cleaned_trans[cleaned_trans['sales_amt'] < 0])

# Create comparison dataframe
metrics = {
    'Metric': ['Missing Values', 'Negative Sales'],
    'Before': [
        original_missing_count,
        original_neg_sales_count,
    ],
    'After': [
        current_missing_count,
        current_neg_sales_count,
    ]
}
metrics_df = pd.DataFrame(metrics)
metrics_df['Improvement'] = metrics_df['Before'] - metrics_df['After']
metrics_df['% Change'] = (metrics_df['Improvement'] / metrics_df['Before'] * 100).round(2)

# Display comparison table
print("\nData Quality Improvements:")
print(metrics_df.to_string(index=False))

# Visualize the improvements
plt.figure(figsize=(12, 7))
metrics_df.set_index('Metric', inplace=True)
ax = metrics_df[['Before', 'After']].plot(kind='bar', color=['#ff7f0e', '#1f77b4'])
for i, v in enumerate(metrics_df['Before']):
    ax.text(i-0.15, v/2, str(v), color='white', fontweight='bold', ha='center')
for i, v in enumerate(metrics_df['After']):
    ax.text(i+0.15, v/2, str(v), color='white', fontweight='bold', ha='center')
plt.title('Data Quality Improvements', fontsize=16)
plt.ylabel('Count', fontsize=14)
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig('visualizations/data_quality_improvements.png')
plt.show()

# 5. Flagged Anomalies Visualization
anomaly_counts = pd.Series({
    'Returns': cleaned_trans['is_return'].sum() if 'is_return' in cleaned_trans.columns else 0,
    'Outlier Customers': cleaned_trans['is_outlier_customer'].sum() if 'is_outlier_customer' in cleaned_trans.columns else 0,
    'Low Activity Stores': cleaned_trans['is_low_activity_store'].sum() if 'is_low_activity_store' in cleaned_trans.columns else 0,
    'Missing Product Info': cleaned_trans['has_missing_product_info'].sum() if 'has_missing_product_info' in cleaned_trans.columns else 0
})

plt.figure(figsize=(10, 6))
ax = anomaly_counts.plot(kind='bar', color='purple')
for i, v in enumerate(anomaly_counts):
    ax.text(i, v + 0.1, f"{v:,}\n({v/len(cleaned_trans)*100:.1f}%)", ha='center')
plt.title('Flagged Anomalies in Cleaned Dataset', fontsize=16)
plt.ylabel('Number of Transactions', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('visualizations/flagged_anomalies.png')
plt.show()

print("Post-cleaning visualizations created and displayed")

# 8. Key Business Insights

In [ ]:
print("\n=== Key Business Insights ===")

# 1. Top customers by revenue
print("\n--- Top Customers Analysis ---")
top_customers = customer_stats.sort_values('total_spent', ascending=False).head(10)
print("Top 10 Customers by Total Spend:")
for i, (cust_id, row) in enumerate(top_customers.iterrows(), 1):
    print(f"{i}. Customer {cust_id}: ${row['total_spent']:,.2f} total spent, {row['num_transactions']:,} transactions, {row['unique_products']:,} unique products")

# Visualize top customers
plt.figure(figsize=(12, 6))
plt.bar(range(1, 11), top_customers['total_spent'], color='blue')
plt.title('Top 10 Customers by Total Spend', fontsize=16)
plt.xlabel('Customer Rank', fontsize=14)
plt.ylabel('Total Spend ($)', fontsize=14)
plt.xticks(range(1, 11))
for i, v in enumerate(top_customers['total_spent']):
    plt.text(i+1, v, f"${v:,.0f}", ha='center', va='bottom')
plt.tight_layout()
plt.savefig('visualizations/top_customers.png')
plt.show()

# 2. Top products and categories
print("\n--- Top Products and Categories Analysis ---")

# Join transactions with product info
trans_with_products = cleaned_trans.merge(
    cleaned_prods[['prod_id', 'prod_desc', 'prod_category', 'prod_subcategory']], 
    on='prod_id', 
    how='left'
)

# Top products by sales
product_sales = trans_with_products.groupby(['prod_id', 'prod_desc'])['sales_amt'].sum().sort_values(ascending=False)
top_products = product_sales.head(10)

print("Top 10 Products by Sales Amount:")
for i, ((prod_id, prod_desc), sales) in enumerate(top_products.items(), 1):
    print(f"{i}. Product {prod_id} ({prod_desc}): ${sales:,.2f}")

# Visualize top products
plt.figure(figsize=(14, 7))
top_products_df = product_sales.head(10).reset_index()
plt.barh(top_products_df['prod_desc'].str[:30], top_products_df['sales_amt'], color='green')
plt.title('Top 10 Products by Sales Amount', fontsize=16)
plt.xlabel('Sales Amount ($)', fontsize=14)
plt.ylabel('Product Description', fontsize=14)
plt.gca().invert_yaxis()  # Display highest at top
for i, v in enumerate(top_products_df['sales_amt']):
    plt.text(v, i, f" ${v:,.0f}", va='center')
plt.tight_layout()
plt.savefig('visualizations/top_products.png')
plt.show()

# Top categories by sales (if category information is available)
if 'prod_category' in trans_with_products.columns:
    category_sales = trans_with_products.groupby('prod_category')['sales_amt'].sum().sort_values(ascending=False)
    top_categories = category_sales.head(10)
    
    print("\nTop 10 Categories by Sales Amount:")
    for i, (category, sales) in enumerate(top_categories.items(), 1):
        print(f"{i}. {category}: ${sales:,.2f}")
    
    # Visualize top categories
    plt.figure(figsize=(14, 7))
    plt.barh(top_categories.index, top_categories.values, color='purple')
    plt.title('Top 10 Categories by Sales Amount', fontsize=16)
    plt.xlabel('Sales Amount ($)', fontsize=14)
    plt.ylabel('Product Category', fontsize=14)
    plt.gca().invert_yaxis()  # Display highest at top
    for i, v in enumerate(top_categories.values):
        plt.text(v, i, f" ${v:,.0f}", va='center')
    plt.tight_layout()
    plt.savefig('visualizations/top_categories.png')
    plt.show()

# 3. Store analysis
print("\n--- Store Performance Analysis ---")
store_performance = cleaned_trans.groupby('store_id').agg({
    'sales_amt': 'sum',
    'trans_id': 'nunique',
    'cust_id': 'nunique',
    'prod_id': 'nunique'
})
store_performance.columns = ['total_sales', 'num_transactions', 'unique_customers', 'unique_products']
store_performance['avg_transaction_value'] = store_performance['total_sales'] / store_performance['num_transactions']

top_stores = store_performance.sort_values('total_sales', ascending=False).head(10)
print("Top 10 Stores by Sales Amount:")
for i, (store_id, row) in enumerate(top_stores.iterrows(), 1):
    print(f"{i}. Store {store_id}: ${row['total_sales']:,.2f} total sales, {row['num_transactions']:,} transactions")

# Visualize top stores
plt.figure(figsize=(12, 6))
plt.bar(top_stores.index.astype(str), top_stores['total_sales'], color='orange')
plt.title('Top 10 Stores by Sales Amount', fontsize=16)
plt.xlabel('Store ID', fontsize=14)
plt.ylabel('Total Sales ($)', fontsize=14)
plt.xticks(rotation=45)
for i, v in enumerate(top_stores['total_sales']):
    plt.text(i, v, f"${v:,.0f}", ha='center', va='bottom')
plt.tight_layout()
plt.savefig('visualizations/top_stores.png')
plt.show()

# 4. Temporal analysis
print("\n--- Sales Trends Analysis ---")
# Group by month
monthly_sales = cleaned_trans.groupby(cleaned_trans['trans_dt'].dt.to_period('M'))['sales_amt'].sum()
monthly_sales.index = monthly_sales.index.astype(str)

# Visualize monthly sales
plt.figure(figsize=(14, 7))
plt.plot(monthly_sales.index, monthly_sales.values, marker='o', linewidth=2, color='blue')
plt.title('Monthly Sales Trends', fontsize=16)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Total Sales ($)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
for i, v in enumerate(monthly_sales.values):
    plt.text(i, v, f"${v:,.0f}", ha='center', va='bottom')
plt.tight_layout()
plt.savefig('visualizations/monthly_sales.png')
plt.show()

print("Key business insights analysis complete. Visualizations saved to 'visualizations' directory.")

In [ ]:
# Save the final cleaned datasets with appropriate names
cleaned_trans.to_csv('acse_transactions_cleaned_final.csv', index=False)
cleaned_prods.to_csv('acse_products_cleaned_final.csv', index=False)

print("\n=== Final Cleaned Datasets Saved ===")
print(f"• Cleaned transactions dataset saved to 'acse_transactions_cleaned_final.csv' ({len(cleaned_trans):,} rows)")
print(f"• Cleaned products dataset saved to 'acse_products_cleaned_final.csv' ({len(cleaned_prods):,} rows)")
print("• All detected anomalies are flagged rather than removed, preserving data integrity")
print("• The datasets are now ready for further analysis and building the recommender system")